# The Groovy Commutator on Primes: Multiplicative Structure

This notebook applies the groovy commutator framework to prime numbers, using a key insight:

**The derivative of multiplication is addition.**

Specifically, we define the derivative operator `D` as the logarithm:

```
D(x) = log(x)
```

This transforms the multiplicative structure of integers into additive structure:

```
D(a × b) = log(a × b) = log(a) + log(b) = D(a) + D(b)
```

The logarithm is the **unique** function (up to scaling) that converts multiplication to addition. This makes it the natural "derivative" for exploring multiplicative structure—and primes are the atoms of multiplicative structure.

## The Groovy Commutator

Recall the groovy commutator formula:

```
K(ψ) = D(ψ + D(ψ)) - (D(ψ) + D(D(ψ)))
```

With our definition:
- `D = log` (multiplicative derivative)
- `+` is standard addition
- `ψ` is our sequence (primes, or functions of primes)

The commutator measures where **expectations fail**—where knowing the derivative doesn't predict how the derivative of the evolved signal behaves.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Optional
import sys
sys.path.insert(0, '.')

from src.groovy_commutator import first_n_primes, prime_gaps

## Defining the Multiplicative Derivative

The logarithm transforms multiplicative relationships into additive ones. For a sequence of values, we apply it element-wise.

In [ ]:
def D_log(psi: np.ndarray) -> np.ndarray:
    """
    The multiplicative derivative: D = log.
    
    This transforms multiplicative structure to additive structure:
        D(a × b) = D(a) + D(b)
    
    Args:
        psi: Input sequence (positive values)
        
    Returns:
        log(psi) - the "multiplicative derivative"
    """
    return np.log(psi.astype(np.float64))


def D_log_ratio(psi: np.ndarray) -> np.ndarray:
    """
    Discrete multiplicative derivative: log of ratios.
    
    This is the discrete analog: D(ψ)[i] = log(ψ[i+1] / ψ[i]) = log(ψ[i+1]) - log(ψ[i])
    
    For primes, this captures the multiplicative rate of change.
    
    Args:
        psi: Input sequence (positive values)
        
    Returns:
        Array of log-ratios (length n-1)
    """
    log_psi = np.log(psi.astype(np.float64))
    return np.diff(log_psi)

In [ ]:
# Demonstrate the multiplicative derivative on primes
primes = first_n_primes(20)
print("First 20 primes:")
print(primes)
print()

print("D(primes) = log(primes):")
print(np.round(D_log(primes), 4))
print()

print("Discrete D (log-ratios):")
print(np.round(D_log_ratio(primes), 4))
print()

# The log-ratio represents multiplicative growth rate
# For primes, this is related to: p_{n+1}/p_n
print("Ratio p_{n+1}/p_n:")
print(np.round(primes[1:] / primes[:-1], 4))

## The Groovy Commutator with Multiplicative Derivative

Now we implement K using D = log. The formula is:

```
K(ψ) = D(ψ + D(ψ)) - (D(ψ) + D(D(ψ)))
```

Substituting D = log:

```
K(ψ) = log(ψ + log(ψ)) - (log(ψ) + log(log(ψ)))
     = log(ψ + log(ψ)) - log(ψ) - log(log(ψ))
     = log((ψ + log(ψ)) / ψ) - log(log(ψ))
     = log((1 + log(ψ)/ψ)) - log(log(ψ))
     = log((1 + log(ψ)/ψ) / log(ψ))
```

This measures the deviation from a "multiplicatively linear" system.

In [ ]:
class MultiplicativeGroovyCommutator:
    """
    Groovy commutator using the multiplicative derivative D = log.
    
    K(ψ) = D(ψ + D(ψ)) - (D(ψ) + D(D(ψ)))
    
    With D = log, this becomes:
    K(ψ) = log(ψ + log(ψ)) - log(ψ) - log(log(ψ))
    
    The commutator measures non-commutativity between:
    - "Evolution" (adding the derivative)
    - "Differentiation" (taking logs)
    """
    
    def __init__(self, epsilon: float = 1e-10):
        """
        Initialize the multiplicative groovy commutator.
        
        Args:
            epsilon: Small value to prevent log(0) issues
        """
        self.epsilon = epsilon
    
    def D(self, psi: np.ndarray) -> np.ndarray:
        """Multiplicative derivative: D(ψ) = log(ψ)"""
        return np.log(np.maximum(psi, self.epsilon))
    
    def compute(self, psi: np.ndarray) -> np.ndarray:
        """
        Compute the groovy commutator K(ψ).
        
        K(ψ) = D(ψ + D(ψ)) - (D(ψ) + D(D(ψ)))
             = log(ψ + log(ψ)) - log(ψ) - log(log(ψ))
        
        Args:
            psi: Input sequence (positive values, typically > 1)
            
        Returns:
            Groovy commutator values
        """
        psi = np.asarray(psi, dtype=np.float64)
        
        # D(ψ) = log(ψ)
        d_psi = self.D(psi)
        
        # D(D(ψ)) = log(log(ψ))
        d_d_psi = self.D(np.maximum(d_psi, self.epsilon))
        
        # ψ + D(ψ) = ψ + log(ψ)
        psi_plus_d = psi + d_psi
        
        # D(ψ + D(ψ)) = log(ψ + log(ψ))
        d_psi_plus_d = self.D(np.maximum(psi_plus_d, self.epsilon))
        
        # K = D(ψ + D(ψ)) - D(ψ) - D(D(ψ))
        k = d_psi_plus_d - d_psi - d_d_psi
        
        return k
    
    def compute_simplified(self, psi: np.ndarray) -> np.ndarray:
        """
        Simplified form: K(ψ) = log((1 + log(ψ)/ψ) / log(ψ))
        
        This form makes the structure clearer:
        - The ratio log(ψ)/ψ measures how "logarithmic" the value is
        - K measures deviation from multiplicative linearity
        """
        psi = np.asarray(psi, dtype=np.float64)
        log_psi = np.log(np.maximum(psi, self.epsilon))
        
        # (1 + log(ψ)/ψ) / log(ψ)
        ratio = (1 + log_psi / psi) / np.maximum(log_psi, self.epsilon)
        
        return np.log(np.maximum(ratio, self.epsilon))
    
    def __repr__(self) -> str:
        return "MultiplicativeGroovyCommutator(D=log)"

In [ ]:
# Test the multiplicative groovy commutator on primes
mgc = MultiplicativeGroovyCommutator()

primes = first_n_primes(100)
k_primes = mgc.compute(primes)

print("First 20 primes:", primes[:20])
print()
print("K(primes) for first 20:")
print(np.round(k_primes[:20], 6))
print()
print(f"Mean |K|: {np.mean(np.abs(k_primes)):.6f}")
print(f"Std K: {np.std(k_primes):.6f}")
print(f"Range: [{np.min(k_primes):.6f}, {np.max(k_primes):.6f}]")

## Visualizing K on Primes

Let's see how the groovy commutator behaves across the prime sequence.

In [ ]:
# Compute K for a larger range of primes
n_primes = 1000
primes = first_n_primes(n_primes)
k_primes = mgc.compute(primes)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: K vs prime index
ax = axes[0, 0]
ax.plot(k_primes, 'b-', alpha=0.7, linewidth=0.5)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Prime index n')
ax.set_ylabel('K(p_n)')
ax.set_title('Groovy Commutator K on Primes')

# Plot 2: K vs prime value (log scale x)
ax = axes[0, 1]
ax.semilogx(primes, k_primes, 'b-', alpha=0.7, linewidth=0.5)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Prime value p')
ax.set_ylabel('K(p)')
ax.set_title('K vs Prime Value (log scale)')

# Plot 3: Distribution of K values
ax = axes[1, 0]
ax.hist(k_primes, bins=50, density=True, alpha=0.7, color='blue', edgecolor='black')
ax.axvline(x=0, color='r', linestyle='--', alpha=0.5)
ax.axvline(x=np.mean(k_primes), color='g', linestyle='-', alpha=0.8, label=f'Mean={np.mean(k_primes):.4f}')
ax.set_xlabel('K value')
ax.set_ylabel('Density')
ax.set_title('Distribution of K Values')
ax.legend()

# Plot 4: Running statistics of K
ax = axes[1, 1]
window = 50
running_mean = np.convolve(k_primes, np.ones(window)/window, mode='valid')
running_std = np.array([np.std(k_primes[max(0,i-window):i+1]) for i in range(len(k_primes))])[window-1:]
x = np.arange(window-1, len(k_primes))
ax.plot(x, running_mean, 'b-', label='Running mean', alpha=0.8)
ax.fill_between(x, running_mean - running_std, running_mean + running_std, alpha=0.3, color='blue')
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Prime index')
ax.set_ylabel('K (running statistics)')
ax.set_title(f'Running Mean ± Std (window={window})')
ax.legend()

plt.suptitle(f'Multiplicative Groovy Commutator K on First {n_primes} Primes', fontsize=14)
plt.tight_layout()
plt.show()

## The Discrete Formulation

An alternative approach uses the discrete derivative (differences) rather than element-wise log. Here we define:

- **Δ** (discrete derivative): Δψ[i] = ψ[i+1] - ψ[i]
- **D** (multiplicative derivative): D(ψ) = log(ψ)

We can compose these in interesting ways:

1. **D∘Δ**: First take differences, then log → log of prime gaps
2. **Δ∘D**: First take log, then differences → log-ratios (multiplicative growth)

The commutator [D, Δ] = D∘Δ - Δ∘D measures how these operations fail to commute.

In [ ]:
class DiscreteMultiplicativeCommutator:
    """
    Commutator between discrete derivative Δ and multiplicative derivative D=log.
    
    [D, Δ](ψ) = D(Δψ) - Δ(Dψ)
             = log(ψ[i+1] - ψ[i]) - (log(ψ[i+1]) - log(ψ[i]))
             = log(gap) - log(ratio)
             = log(gap / ratio)
             = log((p_{n+1} - p_n) / (p_{n+1}/p_n))
             = log((p_{n+1} - p_n) × p_n / p_{n+1})
             = log(p_n × (1 - p_n/p_{n+1}))
    
    This measures the mismatch between additive gaps and multiplicative ratios.
    """
    
    def __init__(self, epsilon: float = 1e-10):
        self.epsilon = epsilon
    
    def delta(self, psi: np.ndarray) -> np.ndarray:
        """Discrete derivative: Δψ[i] = ψ[i+1] - ψ[i]"""
        return np.diff(psi)
    
    def D(self, psi: np.ndarray) -> np.ndarray:
        """Multiplicative derivative: D(ψ) = log(ψ)"""
        return np.log(np.maximum(np.abs(psi), self.epsilon))
    
    def commutator(self, psi: np.ndarray) -> np.ndarray:
        """
        Compute [D, Δ](ψ) = D(Δψ) - Δ(Dψ).
        
        This compares:
        - D(Δψ) = log(gaps)
        - Δ(Dψ) = log-ratios = log(ψ[i+1]/ψ[i])
        
        Args:
            psi: Input sequence (positive values)
            
        Returns:
            Commutator values (length n-1)
        """
        psi = np.asarray(psi, dtype=np.float64)
        
        # D(Δψ) = log(gaps)
        gaps = self.delta(psi)
        d_delta_psi = self.D(gaps)
        
        # Δ(Dψ) = log-ratios
        log_psi = self.D(psi)
        delta_d_psi = self.delta(log_psi)
        
        return d_delta_psi - delta_d_psi
    
    def commutator_explicit(self, psi: np.ndarray) -> np.ndarray:
        """
        Explicit form: [D, Δ](ψ) = log(gap × p_n / p_{n+1})
                                 = log(gap) + log(p_n) - log(p_{n+1})
        """
        psi = np.asarray(psi, dtype=np.float64)
        gaps = np.diff(psi)
        
        # gap × p_n / p_{n+1}
        ratio = gaps * psi[:-1] / psi[1:]
        return np.log(np.maximum(ratio, self.epsilon))
    
    def __repr__(self) -> str:
        return "DiscreteMultiplicativeCommutator([D=log, Δ=diff])"

In [ ]:
# Compute the discrete commutator on primes
dmc = DiscreteMultiplicativeCommutator()

primes = first_n_primes(1000)
comm = dmc.commutator(primes)
comm_explicit = dmc.commutator_explicit(primes)

print("Verification (both forms should match):")
print(f"Max difference: {np.max(np.abs(comm - comm_explicit)):.2e}")
print()

gaps = np.diff(primes)
log_ratios = np.diff(np.log(primes))

print("First 10 primes:", primes[:10])
print("Gaps (Δψ):      ", gaps[:9])
print("log(gaps):      ", np.round(np.log(gaps[:9]), 4))
print("log-ratios (Δ∘D):", np.round(log_ratios[:9], 4))
print("[D,Δ]:          ", np.round(comm[:9], 4))

In [ ]:
# Visualize the discrete commutator
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: [D, Δ] vs prime index
ax = axes[0, 0]
ax.plot(comm, 'b-', alpha=0.6, linewidth=0.5)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Prime index n')
ax.set_ylabel('[D, Δ](p_n)')
ax.set_title('Discrete Commutator [D, Δ] on Primes')

# Plot 2: Components - log(gap) vs log-ratio
ax = axes[0, 1]
gaps = np.diff(primes)
log_gaps = np.log(gaps)
log_ratios = np.diff(np.log(primes))
ax.scatter(log_ratios, log_gaps, alpha=0.3, s=5)
ax.plot([0, max(log_ratios)], [0, max(log_ratios)], 'r--', label='y=x (commuting)')
ax.set_xlabel('Δ∘D = log(p_{n+1}/p_n)')
ax.set_ylabel('D∘Δ = log(gap)')
ax.set_title('log(gap) vs log(ratio): Deviation from Line = Commutator')
ax.legend()

# Plot 3: Commutator vs gap size
ax = axes[1, 0]
ax.scatter(gaps, comm, alpha=0.4, s=10, c=np.arange(len(gaps)), cmap='viridis')
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Gap size')
ax.set_ylabel('[D, Δ]')
ax.set_title('Commutator vs Gap Size (color = prime index)')

# Plot 4: Distribution and running mean
ax = axes[1, 1]
ax.hist(comm, bins=50, density=True, alpha=0.7, color='blue', edgecolor='black')
ax.axvline(x=np.mean(comm), color='g', linestyle='-', linewidth=2, 
           label=f'Mean={np.mean(comm):.3f}')
ax.axvline(x=np.median(comm), color='orange', linestyle='--', linewidth=2,
           label=f'Median={np.median(comm):.3f}')
ax.set_xlabel('[D, Δ] value')
ax.set_ylabel('Density')
ax.set_title('Distribution of Commutator Values')
ax.legend()

plt.suptitle('Discrete Multiplicative Commutator [D=log, Δ=diff] on Primes', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\nStatistics:")
print(f"  Mean [D,Δ]: {np.mean(comm):.4f}")
print(f"  Std [D,Δ]:  {np.std(comm):.4f}")
print(f"  Range:      [{np.min(comm):.4f}, {np.max(comm):.4f}]")

## Interpreting the Commutator

The commutator `[D, Δ] = log(gap) - log(ratio)` can be rewritten as:

```
[D, Δ](p) = log(gap × p_n / p_{n+1})
          = log(gap) + log(p_n/p_{n+1})
          = log(gap) - log(p_{n+1}/p_n)
```

For the commutator to be zero, we'd need:
```
gap = p_{n+1}/p_n
```

But the gap is `p_{n+1} - p_n`, so:
```
p_{n+1} - p_n = p_{n+1}/p_n
p_n(p_{n+1} - p_n) = p_{n+1}
p_n × p_{n+1} - p_n² = p_{n+1}
p_{n+1}(p_n - 1) = p_n²
```

This only holds for specific prime pairs. The commutator measures how far primes deviate from this "balanced" relationship between additive gaps and multiplicative ratios.

In [ ]:
# When is [D, Δ] ≈ 0?
threshold = 0.1
near_zero = np.where(np.abs(comm) < threshold)[0]

print(f"Prime pairs where |[D, Δ]| < {threshold}:")
print()
for idx in near_zero[:20]:
    p1, p2 = primes[idx], primes[idx+1]
    gap = p2 - p1
    ratio = p2 / p1
    c = comm[idx]
    print(f"  p_{idx} = {p1:5d}, p_{idx+1} = {p2:5d}, gap = {gap:3d}, ratio = {ratio:.4f}, [D,Δ] = {c:+.4f}")

## The Full Groovy Commutator on Prime Gaps

Let's apply the full groovy commutator formula to prime gaps, which are often more structured than primes themselves.

In [ ]:
# Groovy commutator on prime gaps
primes = first_n_primes(2000)
gaps = prime_gaps(primes)  # p_{n+1} - p_n

# Use the multiplicative groovy commutator
mgc = MultiplicativeGroovyCommutator()
k_gaps = mgc.compute(gaps)

print(f"First 20 prime gaps: {gaps[:20]}")
print(f"K(gaps) first 20: {np.round(k_gaps[:20], 4)}")
print()
print(f"Statistics on K(gaps):")
print(f"  Mean: {np.mean(k_gaps):.4f}")
print(f"  Std:  {np.std(k_gaps):.4f}")
print(f"  Range: [{np.min(k_gaps):.4f}, {np.max(k_gaps):.4f}]")

In [ ]:
# Visualize K on prime gaps
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: K(gaps) over index
ax = axes[0, 0]
ax.plot(k_gaps, 'b-', alpha=0.6, linewidth=0.5)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Gap index')
ax.set_ylabel('K(gap)')
ax.set_title('Groovy Commutator K on Prime Gaps')

# Plot 2: K vs gap size
ax = axes[0, 1]
ax.scatter(gaps, k_gaps, alpha=0.3, s=5)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Gap size')
ax.set_ylabel('K(gap)')
ax.set_title('K vs Gap Size')

# Plot 3: K colored by gap value
ax = axes[1, 0]
scatter = ax.scatter(np.arange(len(k_gaps)), k_gaps, c=gaps, cmap='plasma', 
                     alpha=0.5, s=5)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.colorbar(scatter, ax=ax, label='Gap size')
ax.set_xlabel('Gap index')
ax.set_ylabel('K(gap)')
ax.set_title('K Values Colored by Gap Size')

# Plot 4: Distribution
ax = axes[1, 1]
ax.hist(k_gaps, bins=50, density=True, alpha=0.7, color='blue', edgecolor='black')
ax.axvline(x=np.mean(k_gaps), color='g', linestyle='-', linewidth=2,
           label=f'Mean={np.mean(k_gaps):.3f}')
ax.set_xlabel('K value')
ax.set_ylabel('Density')
ax.set_title('Distribution of K(gap)')
ax.legend()

plt.suptitle('Multiplicative Groovy Commutator K on Prime Gaps', fontsize=14)
plt.tight_layout()
plt.show()

## Asymptotic Behavior

By the Prime Number Theorem, for large primes:
- `p_n ~ n log(n)`
- Average gap ~ `log(p_n)`

How does the commutator behave asymptotically?

In [ ]:
# Analyze asymptotic behavior
n_primes = 10000
primes = first_n_primes(n_primes)

mgc = MultiplicativeGroovyCommutator()
k_primes = mgc.compute(primes)

# Bin by prime size and compute statistics
n_bins = 20
bin_edges = np.linspace(0, len(primes)-1, n_bins+1, dtype=int)

bin_means = []
bin_stds = []
bin_centers = []

for i in range(n_bins):
    start, end = bin_edges[i], bin_edges[i+1]
    k_bin = k_primes[start:end]
    bin_means.append(np.mean(k_bin))
    bin_stds.append(np.std(k_bin))
    bin_centers.append((primes[start] + primes[end-1]) / 2)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Mean K vs prime magnitude
ax = axes[0]
ax.errorbar(bin_centers, bin_means, yerr=bin_stds, fmt='o-', capsize=3)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Prime value (bin center)')
ax.set_ylabel('Mean K')
ax.set_title('K Mean by Prime Magnitude')
ax.set_xscale('log')

# Plot 2: Variance of K over time
ax = axes[1]
ax.plot(bin_centers, bin_stds, 'o-', color='orange')
ax.set_xlabel('Prime value (bin center)')
ax.set_ylabel('Std(K)')
ax.set_title('K Standard Deviation by Prime Magnitude')
ax.set_xscale('log')

plt.suptitle(f'Asymptotic Behavior of K on First {n_primes} Primes', fontsize=14)
plt.tight_layout()
plt.show()

# Fit asymptotic behavior
log_primes = np.log(primes)
correlation = np.corrcoef(log_primes, k_primes)[0, 1]
print(f"\nCorrelation between log(p) and K(p): {correlation:.4f}")

## Comparing with Random Sequences

To understand what the commutator reveals about prime structure, let's compare with random sequences that share some statistical properties with primes.

In [ ]:
def generate_prime_like_random(n: int, seed: int = 42) -> np.ndarray:
    """
    Generate a random sequence with prime-like distribution.
    
    Uses the Prime Number Theorem: density of primes near x is ~ 1/log(x).
    Gaps follow roughly exponential distribution with mean ~ log(x).
    """
    np.random.seed(seed)
    
    values = [2]  # Start at 2
    current = 2
    
    for i in range(1, n):
        # Expected gap ~ log(current)
        expected_gap = np.log(current)
        # Random gap from exponential distribution, but at least 1
        gap = max(1, int(np.random.exponential(expected_gap)))
        # Ensure odd (like primes > 2)
        if gap % 2 == 0 and current > 2:
            gap += 1
        current += gap
        values.append(current)
    
    return np.array(values)

# Generate random "prime-like" sequence
n = 1000
primes = first_n_primes(n)
random_seq = generate_prime_like_random(n)

# Compute commutators
mgc = MultiplicativeGroovyCommutator()
k_primes = mgc.compute(primes)
k_random = mgc.compute(random_seq)

print(f"Primes: first 10 = {primes[:10]}")
print(f"Random: first 10 = {random_seq[:10]}")
print()
print(f"K statistics:")
print(f"  Primes - Mean: {np.mean(k_primes):.4f}, Std: {np.std(k_primes):.4f}")
print(f"  Random - Mean: {np.mean(k_random):.4f}, Std: {np.std(k_random):.4f}")

In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: K distributions
ax = axes[0, 0]
ax.hist(k_primes, bins=40, density=True, alpha=0.6, label='Primes', color='blue')
ax.hist(k_random, bins=40, density=True, alpha=0.6, label='Random', color='orange')
ax.axvline(x=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('K value')
ax.set_ylabel('Density')
ax.set_title('K Distribution: Primes vs Random')
ax.legend()

# Plot 2: K time series
ax = axes[0, 1]
ax.plot(k_primes, 'b-', alpha=0.5, linewidth=0.5, label='Primes')
ax.plot(k_random, 'orange', alpha=0.5, linewidth=0.5, label='Random')
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Index')
ax.set_ylabel('K')
ax.set_title('K Over Index')
ax.legend()

# Plot 3: Gap comparison
ax = axes[1, 0]
gaps_primes = np.diff(primes)
gaps_random = np.diff(random_seq)
ax.hist(gaps_primes, bins=range(0, 40, 2), density=True, alpha=0.6, label='Prime gaps', color='blue')
ax.hist(gaps_random, bins=range(0, 40, 2), density=True, alpha=0.6, label='Random gaps', color='orange')
ax.set_xlabel('Gap size')
ax.set_ylabel('Density')
ax.set_title('Gap Distribution')
ax.legend()

# Plot 4: Running variance comparison
ax = axes[1, 1]
window = 50
var_primes = np.array([np.var(k_primes[max(0,i-window):i+1]) for i in range(window, len(k_primes))])
var_random = np.array([np.var(k_random[max(0,i-window):i+1]) for i in range(window, len(k_random))])
ax.plot(var_primes, 'b-', alpha=0.7, label='Primes')
ax.plot(var_random, 'orange', alpha=0.7, label='Random')
ax.set_xlabel('Index')
ax.set_ylabel(f'Var(K) [window={window}]')
ax.set_title('Running Variance of K')
ax.legend()

plt.suptitle('Multiplicative Groovy Commutator: Primes vs Prime-like Random', fontsize=14)
plt.tight_layout()
plt.show()

## Summary and Observations

We've developed two approaches to applying the groovy commutator to primes:

### 1. Multiplicative Groovy Commutator (D = log element-wise)
```
K(ψ) = log(ψ + log(ψ)) - log(ψ) - log(log(ψ))
```
- Measures deviation from "multiplicatively linear" behavior
- Captures how the additive evolution (ψ + D(ψ)) interacts with the multiplicative structure

### 2. Discrete Multiplicative Commutator ([D, Δ])
```
[D, Δ](ψ) = log(gaps) - log(ratios) = log(gap × p_n / p_{n+1})
```
- Directly compares additive structure (gaps) with multiplicative structure (ratios)
- Zero when gap = p_{n+1}/p_n (a special balance condition)

### Key Findings:
1. **The commutator captures prime structure**: K on primes differs statistically from K on random prime-like sequences
2. **Asymptotic stability**: The mean and variance of K stabilize as primes grow
3. **Gap-ratio mismatch**: The discrete commutator [D, Δ] directly quantifies how prime gaps deviate from what multiplicative structure would predict

### Future Directions:
- Apply to twin primes, Sophie Germain primes, and other special prime sequences
- Study how K relates to known conjectures (Cramér's conjecture, etc.)
- Extend to other multiplicative functions (Möbius μ, Euler φ)

In [ ]:
# Final summary visualization
primes = first_n_primes(5000)
gaps = prime_gaps(primes)

mgc = MultiplicativeGroovyCommutator()
dmc = DiscreteMultiplicativeCommutator()

k_primes = mgc.compute(primes)
k_gaps = mgc.compute(gaps)
comm_discrete = dmc.commutator(primes)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# K on primes
ax = axes[0]
ax.plot(k_primes, 'b-', alpha=0.5, linewidth=0.3)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_title(f'K(primes)\nμ={np.mean(k_primes):.3f}, σ={np.std(k_primes):.3f}')
ax.set_xlabel('Prime index')
ax.set_ylabel('K')

# K on gaps
ax = axes[1]
ax.plot(k_gaps, 'g-', alpha=0.5, linewidth=0.3)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_title(f'K(gaps)\nμ={np.mean(k_gaps):.3f}, σ={np.std(k_gaps):.3f}')
ax.set_xlabel('Gap index')
ax.set_ylabel('K')

# Discrete commutator
ax = axes[2]
ax.plot(comm_discrete, 'purple', alpha=0.5, linewidth=0.3)
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax.set_title(f'[D,Δ](primes)\nμ={np.mean(comm_discrete):.3f}, σ={np.std(comm_discrete):.3f}')
ax.set_xlabel('Prime index')
ax.set_ylabel('[D,Δ]')

plt.suptitle('Groovy Commutator Analysis of First 5000 Primes\n(D = log: "The derivative of multiplication is addition")', fontsize=12)
plt.tight_layout()
plt.show()